In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Activation, Input, concatenate, Reshape, Flatten
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd /content/drive/MyDrive/Optimal Price Detection

/content/drive/MyDrive/Optimal Price Detection


In [5]:
data = pd.read_csv('data.csv')
price = pd.read_csv('daily_price.csv').iloc[:, 1:]
coef = pd.read_csv('booking_ratio.csv').iloc[:, 1:]

In [6]:
benchmark_df = pd.read_csv('benchmark_data.csv')

In [7]:
# Encoding size column
weights = {'small': 1, 'medium':2, 'large':3}
data['Size'] = data['Size'].replace(weights)
benchmark_df['Size'] = benchmark_df['Size'].replace(weights)

In [8]:
X_lstm_benchmark = benchmark_df.iloc[:, 4:].applymap(func=lambda row: float(eval(row)[0])).astype(float)
X_dense_benchmark = benchmark_df.iloc[:, :4]

X_lstm_benchmark = X_lstm_benchmark.to_numpy().reshape(X_lstm_benchmark.shape[0], 520, 1)
X_dense_benchmark = X_dense_benchmark.to_numpy().reshape(X_dense_benchmark.shape[0], 4, 1)

X_lstm_benchmark = torch.tensor(X_lstm_benchmark).float()
X_dense_benchmark = torch.tensor(X_dense_benchmark).float()

In [10]:
X_lstm_benchmark = torch.tensor(X_lstm_benchmark)
X_dense_benchmark = torch.tensor(X_dense_benchmark)

<ipython-input-10-9897c84e10ba>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_lstm_benchmark = torch.tensor(X_lstm_benchmark)
<ipython-input-10-9897c84e10ba>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_dense_benchmark = torch.tensor(X_dense_benchmark)


In [12]:
# Dense data
X_train_dense, X_test_dense, _ , _ = train_test_split(data.iloc[:, :4], coef)

# Normalizing, before converting to numpy array
lat_long_scaler = MinMaxScaler(feature_range=(-1, 1))
X_train_dense.iloc[:, 0] = lat_long_scaler.fit_transform(X_train_dense.iloc[:, 0].values.reshape(-1, 1))
X_train_dense.iloc[:, 1] = lat_long_scaler.fit_transform(X_train_dense.iloc[:, 0].values.reshape(-1, 1))

X_train_dense = X_train_dense.to_numpy().reshape(X_train_dense.shape[0], 4, 1)
X_test_dense = X_test_dense.to_numpy().reshape(X_test_dense.shape[0], 4, 1)

In [13]:
# LSTM data
X_train, X_test, y_train, y_test = train_test_split(price, coef)

# Normalizing, before converting to numpy array
scaler = MinMaxScaler()
X_train_lstm = scaler.fit_transform(X_train.values)

X_train_lstm = X_train.to_numpy().reshape(X_train.shape[0], 520, 1)
y_train = y_train.to_numpy().reshape(y_train.shape[0], 520, 1)
X_test_lstm = X_test.to_numpy().reshape(X_test.shape[0], 520, 1)
y_test = y_test.to_numpy().reshape(y_test.shape[0], 520, 1)


In [14]:
X_train_lstm = torch.tensor(X_train_lstm).float()
X_train_dense = torch.tensor(X_train_dense).float()
y_train = torch.tensor(y_train).float()
y_test = torch.tensor(y_test).float()
X_test_lstm = torch.tensor(X_test_lstm).float()
X_test_dense = torch.tensor(X_test_dense).float()

In [44]:
# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size=520, hidden_size=260, num_layers=1, batch_first=True)
        self.activation1 = nn.Sigmoid()
    def forward(self, x):
        out, _ = self.lstm1(x)
        out = self.activation1(out)
        return out[:, -1, :]  # Taking only the last time step's output
# Define the dense neural network
class DenseModel(nn.Module):
    def __init__(self):
        super(DenseModel, self).__init__()
        self.dense1 = nn.Linear(1, 4)
        self.activation1 = nn.ReLU()
        self.dense2 = nn.Linear(4, 128)
        self.activation2 = nn.ReLU()
        self.dense3 = nn.Linear(128, 128)
        self.activation3 = nn.ReLU()
        self.dense4 = nn.Linear(128, 260)
    def forward(self, x):
        out = self.dense1(x)
        out = self.activation1(out)
        out = self.dense2(out)
        out = self.activation2(out)
        out = self.dense3(out)
        out = self.activation3(out)
        out = self.dense4(out)
        return out[:, -1, :]
class CombinedModel(nn.Module):
    def __init__(self):
        super(CombinedModel, self).__init__()
        self.lstm_model = LSTMModel()
        self.dense_model = DenseModel()
    def forward(self, lstm_input, dense_input):
        lstm_output = self.lstm_model(lstm_input)
        dense_output = self.dense_model(dense_input)
        combined_output = torch.cat((lstm_output, dense_output), dim=1)
        return combined_output
    def evaluate(self, inputs, labels):
        lstm_input, dense_input = inputs
        outputs = self.forward(lstm_input, dense_input)
        loss = criterion(outputs, labels.squeeze(dim=2))
        return outputs, loss
    def predict(self, inputs):
        lstm_input, dense_input = inputs
        outputs = self.forward(lstm_input, dense_input)
        return outputs

combined_model = CombinedModel()
criterion = nn.MSELoss()
optimizer = optim.Adam(combined_model.parameters(), lr=0.001)

# batch_size = 64
# num_epochs = 30

# for epoch in range(num_epochs):
#     for i in range(0, len(X_train_lstm), batch_size):
#         lstm_batch = X_train_lstm[i:i+batch_size]
#         dense_batch = X_train_dense[i:i+batch_size]
#         labels_batch = y_train[i:i+batch_size]
#         optimizer.zero_grad()
#         outputs = combined_model(lstm_batch, dense_batch)
#         print(f'{i}/7500')
#         loss = criterion(outputs, labels_batch.squeeze(dim=2))
#         loss.backward()
#         optimizer.step()
#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [17]:
# Uploading our trained model
loaded_model = CombinedModel()
loaded_model.load_state_dict(torch.load('combined_model_0-1240-loss-best.pth'))

<All keys matched successfully>

In [18]:
prediction, loss = loaded_model.evaluate([X_lstm_benchmark, X_dense_benchmark], torch.Tensor(2000, 520, 1))

In [32]:
generated_prices = np.zeros((2000,520))
for i in range(520):
    generated_prices[::, i] = 100 / 520 * i

optimal_prices = (prediction * torch.Tensor(generated_prices)).detach().numpy().argmax(axis=1)

In [37]:
result = []
for generated_price, index in zip(generated_prices, optimal_prices):
    optimal_price = generated_price[index]
    result.append(optimal_price)

In [50]:
result = np.array(result)
result

array([91.15384615, 99.42307692, 99.42307692, ..., 91.15384615,
       99.42307692, 99.42307692])

In [51]:
np.save('optimal_prices.npy', result)
np.savetxt('optimal_prices.txt', result)